In [1]:
import torch
import torch.nn as nn
from models.yolo import Model 

In [2]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = torch.mean((outputs - targets)**2)
        return loss
    
config_path = 'models/yolov5s.yaml'

model = Model(config_path)
model.load_state_dict(torch.load('yolov5s.pt')['model'].state_dict())  # yolov5s.ptは、学習済みの重みファイル

print('last lyer type', type(model.model[-1]))
print("number of detection layers", model.model[-1].nl)
print("number of anchors", model.model[-1].na)
print("number of classes", model.model[-1].nc)
print("anchor", model.model[-1].anchors)
print(model.model[-1].m)
print("anchor", len(model.model[-1].anchors[0])//2)



                 from  n    params  module                                  arguments                     


  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  models.common.C3                        [512, 512, 1]                 
  9                -1  1    656896  m

<class 'models.yolo.Detect'>
number of detection layers 3
number of anchors 3
number of classes 80
anchor tensor([[[ 1.25000,  1.62500],
         [ 2.00000,  3.75000],
         [ 4.12500,  2.87500]],

        [[ 1.87500,  3.81250],
         [ 3.87500,  2.81250],
         [ 3.68750,  7.43750]],

        [[ 3.62500,  2.81250],
         [ 4.87500,  6.18750],
         [11.65625, 10.18750]]])
ModuleList(
  (0): Conv2d(128, 255, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 255, kernel_size=(1, 1), stride=(1, 1))
  (2): Conv2d(512, 255, kernel_size=(1, 1), stride=(1, 1))
)
anchor 1


In [4]:
# 最終層を線形層に変更
model.model[-1] = nn.Sequential(*list(model.model[-1].children())[:-1])
print(list(model.model))
# print(model.model.modules.yolo)
num_features = model.model[-1][-1].in_features
num_classes = 10
model.model.add_module('fc', nn.Linear(num_features, num_classes))

# カスタム損失関数のインスタンス化
criterion = CustomLoss()

[Conv(
  (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
), Conv(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
), C3(
  (cv1): Conv(
    (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (cv2): Conv(
    (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (cv3): Conv(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.03,

IndexError: index -1 is out of range